# Data Pre Processing step

In [1]:
from PIL import Image
import cv2 as cv 
import os
import numpy as np
import pandas as pd
import sys
from sklearn.model_selection import train_test_split




In [2]:
# root_folder = os.getcwd()
sys.path.append(os.path.join(os.getcwd()))
os.environ["SARTORIUS"] = os.path.join(os.getcwd())
sart = os.environ["SARTORIUS"]
processed_data_train = os.path.join(sart, 'processed_data','train')
processed_data_test= os.path.join(sart, 'processed_data','test')

In [3]:
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int)
                       for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)


In [4]:
def annotation_conversion(annotation, shape):
    try:
        img_focus = np.zeros(shape)
        img_focus = rle_decode(annotation, shape)
        ycoordinates = np.where(img_focus.sum(axis=1, keepdims=False))
        xcoordinates = np.where(img_focus.sum(axis=0, keepdims=False))
        tuple_dims = (ycoordinates[0][0], ycoordinates[0][-1], xcoordinates[0]
                      [0],    xcoordinates[0][-1])
        return tuple_dims
    except:
        return None


# Creation of Test and Train Datasets for Easy training of CNN

In [6]:
file_path = os.path.join(sart,'sartorius-cell-instance-segmentation', 'train.csv')
df = pd.read_csv(file_path)
shape = (520, 704)

# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


In [7]:
df['tuple_dims']= df.annotation.apply(lambda x: annotation_conversion(x,shape))

In [8]:
df['new_path'] = df.apply(lambda x: x[0] +'_'+ str(x.name)+'.png', axis=1)


In [22]:
train,test = train_test_split(df, test_size=0.15)


In [24]:
def create_subImage(input_file_path, output_file_path, output_file_name, crop_param, shape=None):
    try:
        im = cv.imread(input_file_path)
        one_part = im[:, :, 0]
        left, right, top, bottom = crop_param
        if shape != None:
            r_imaged = cv.resize(
                one_part[left:right, top:bottom], shape, interpolation=cv.INTER_AREA)
        else:
            r_imaged = one_part[left:right, top:bottom]
        if not os.path.exists(output_file_path):
            os.mkdir(output_file_path)
        cv.imwrite(os.path.join(output_file_path, output_file_name), r_imaged)
        return crop_param
    except Exception as E:
        print('Unable to process Image.', str(E))


In [25]:
train.apply(lambda x: create_subImage(os.path.join(sart, 'sartorius-cell-instance-segmentation', 'train', str(x.id) + '.png'),
                                      os.path.join(processed_data_train, str(x.cell_type)), str(x.new_path), x.tuple_dims), axis=1)


58921     (89, 100, 266, 279)
73152    (246, 259, 304, 319)
17226    (406, 422, 567, 590)
1619      (87, 105, 181, 201)
14514    (183, 217, 139, 155)
                 ...         
1004     (135, 152, 674, 692)
61365      (391, 419, 19, 32)
50289    (398, 409, 610, 623)
57751    (472, 486, 263, 284)
9313     (506, 519, 402, 421)
Length: 62547, dtype: object

In [26]:
test.apply(lambda x: create_subImage(os.path.join(sart, 'sartorius-cell-instance-segmentation', 'train', str(x.id) + '.png'),
                                      os.path.join(processed_data_test, str(x.cell_type)), str(x.new_path), x.tuple_dims), axis=1)


38996    (413, 432, 370, 384)
3752     (342, 368, 553, 579)
15954    (487, 506, 348, 372)
61067      (10, 19, 354, 361)
4468     (287, 302, 269, 304)
                 ...         
20387    (383, 410, 101, 182)
16529    (102, 114, 340, 358)
52552        (18, 35, 46, 58)
2573     (316, 384, 672, 702)
33997    (117, 130, 265, 287)
Length: 11038, dtype: object